In [1]:
%load_ext autoreload
%autoreload 2

import os
import json
import pickle
import random
random.seed(42)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import tensorflow as tf
import keras
from keras import layers

import sys
sys.path.append('../src/features')

from subject import Subject

cur_dir = os.getcwd()
abide_dir = os.path.dirname(os.path.dirname(cur_dir)) + '/abide/'
subjects_dir = os.path.dirname(cur_dir) + '/data/ABIDEI_subjects/'
trs_save_file = save_dir = os.path.dirname(cur_dir) + '/data/dicts/ABIDEI_site_trs.json'

In [2]:
# Dictionary with TRs for each scanning site
with open(trs_save_file) as json_file:
    site_trs = json.load(json_file)

In [3]:
# Load ABIDEI preprocessed rois by loading subjects
def open_pickle(f):
    file = open(f,'rb')
    o = pickle.load(file)
    file.close()
    return o

def load_subjects_d(subject_folder):
    subjects_d = {}
    for f in os.listdir(subject_folder):
        s = open_pickle(os.path.join(subject_folder, f))
        subjects_d[s._sub_id] = s
    return subjects_d

In [4]:
subjects_d = load_subjects_d(subjects_dir)
subjects = subjects_d.values()

In [5]:
# For now let's just look at sites with Trs of 2s
clean_subjects = list()
asd_c = 0
for s in subjects:
    if(site_trs[s._site_id] == 2):
        clean_subjects.append(s)
        # Note dx group 1 is positive for ASD
        if(s._label_dict['dx_group'] == 1):
            asd_c += 1

In [6]:
print(f'{asd_c} subjects with ASD out of {len(clean_subjects)} subjects in clean list')

253 subjects with ASD out of 548 subjects in clean list


# Randomly extract sections of even length from scan to use for features
* Doing 3 mins trying to replicate https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5669262/

In [7]:
# Since scans are 2s apart 90 scans is 3 mins
L = 90
# Number of clips per subject
N=10
# Number of ROIs
N_rois = 200
feat_name = 'filt_noglobal_roi_200_Cradd'
def extract_feat_sections(s, feat_name=feat_name, L=L, N=N):
    data = s._data_dict[feat_name]
    feat_secs = list()
    for i in range(N):
        r = int(random.random() * (len(data) - L))
        feat_secs.append(data[r:r+L])
    return np.array(feat_secs)

def create_dataset(subjects, feat_name=feat_name, L=L,N=N):
    X = list()
    Y = list()
    for s in subjects:
        feat_secs = extract_feat_sections(s)
        X.extend(feat_secs)
        # 1 is still classified ASD and 0 is control
        if(s._label_dict['dx_group'] == 1):
        # if(s._sex == 1):
            Y.extend([1]*len(feat_secs))
        else:
            Y.extend([0]*len(feat_secs))
    assert len(X) == len(Y)
    X_ar = np.array(X).reshape(len(X), L, N_rois)
    Y_ar = np.array(Y)
    return X_ar, Y_ar

In [8]:
# In original work 10 fold cross val used with proportion of subjects from each site was approximately the same in all folds
# To start will just randomly split subjects into groups
val_per = .05
test_per = .1
train_subs, val_subs = train_test_split(clean_subjects, test_size=val_per + test_per, random_state=42)
val_subs, test_subs = train_test_split(val_subs, test_size=test_per/(val_per + test_per), random_state=43)
print(f'{len(train_subs)} subjects for training')
print(f'{len(val_subs)} subjects for validation')
print(f'{len(test_subs)} subjects for testing')

465 subjects for training
27 subjects for validation
56 subjects for testing


In [9]:
train_X, train_Y = create_dataset(train_subs)
val_X, val_Y = create_dataset(val_subs)
test_X, test_Y = create_dataset(test_subs)
print(f'{len(train_X)} training examples. {sum(train_Y)} of class 1')
print(f'{len(val_X)} validation examples. {sum(val_Y)} of class 1')
print(f'{len(test_X)} testing examples. {sum(test_Y)} of class 1')

4650 training examples. 2110 of class 1
270 validation examples. 110 of class 1
560 testing examples. 310 of class 1


# Create Transformer model
* Resource: https://keras.io/examples/nlp/text_classification_with_transformer/

In [21]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [37]:
embed_dim = N_rois  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
max_len = L

# inputs = layers.Input(shape=(max_len,))
inputs = layers.Input(shape=(max_len, embed_dim))
# embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
# x = embedding_layer(inputs)
x = inputs
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
# x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model = keras.Model(inputs=inputs, outputs=outputs)
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 90, 200)]         0         
_________________________________________________________________
transformer_block_3 (Transfo (None, 90, 200)           335232    
_________________________________________________________________
dropout_14 (Dropout)         (None, 90, 200)           0         
_________________________________________________________________
dense_14 (Dense)             (None, 90, 20)            4020      
_________________________________________________________________
dropout_15 (Dropout)         (None, 90, 20)            0         
_________________________________________________________________
dense_15 (Dense)             (None, 90, 1)             21        
Total params: 339,273
Trainable params: 339,273
Non-trainable params: 0
_____________________________________________________

In [38]:
# model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(
    train_X, train_Y, batch_size=32, epochs=5, validation_data=(val_X, val_Y)
)

Epoch 1/5
146/146 [==============================] - 58s 383ms/step - loss: 0.7313 - accuracy: 0.5311 - val_loss: 0.6872 - val_accuracy: 0.5660
Epoch 2/5
146/146 [==============================] - 59s 400ms/step - loss: 0.6721 - accuracy: 0.5656 - val_loss: 0.6881 - val_accuracy: 0.5629
Epoch 3/5
146/146 [==============================] - 56s 383ms/step - loss: 0.6293 - accuracy: 0.6427 - val_loss: 0.7296 - val_accuracy: 0.5345
Epoch 4/5
146/146 [==============================] - 57s 390ms/step - loss: 0.5776 - accuracy: 0.6938 - val_loss: 0.7935 - val_accuracy: 0.5581
Epoch 5/5
146/146 [==============================] - 59s 401ms/step - loss: 0.5164 - accuracy: 0.7450 - val_loss: 0.9263 - val_accuracy: 0.5146


In [39]:
# evaluate the model
_, train_acc = model.evaluate(train_X, train_Y, verbose=0)
_, test_acc = model.evaluate(test_X, test_Y, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))

Train: 0.717, Test: 0.505


# Now with a positional embedding layer
* https://keras.io/examples/nlp/masked_language_modeling/

In [40]:
class PositionEmbedding(layers.Layer):
    def __init__(self, maxlen, embed_dim):
        super(PositionEmbedding, self).__init__()
        self.maxlen = maxlen
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        positions = tf.range(start=0, limit=self.maxlen, delta=1)
        positions = self.pos_emb(positions)
        return x + positions

In [44]:
embed_dim = N_rois  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
max_len = L

# inputs = layers.Input(shape=(max_len,))
inputs = layers.Input(shape=(max_len, embed_dim))
pos_embedding_layer = PositionEmbedding(max_len, embed_dim)
x = pos_embedding_layer(inputs)
# x = inputs
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
# x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model_embed = keras.Model(inputs=inputs, outputs=outputs)
model_embed.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 90, 200)]         0         
_________________________________________________________________
position_embedding_3 (Positi (None, 90, 200)           18000     
_________________________________________________________________
transformer_block_6 (Transfo (None, 90, 200)           335232    
_________________________________________________________________
dropout_26 (Dropout)         (None, 90, 200)           0         
_________________________________________________________________
dense_26 (Dense)             (None, 90, 20)            4020      
_________________________________________________________________
dropout_27 (Dropout)         (None, 90, 20)            0         
_________________________________________________________________
dense_27 (Dense)             (None, 90, 1)             21  

In [45]:
model_embed.compile("adam", "binary_crossentropy", metrics=["accuracy"])
history = model_embed.fit(
    train_X, train_Y, batch_size=32, epochs=2, validation_data=(val_X, val_Y)
)

Epoch 1/2
146/146 [==============================] - 71s 459ms/step - loss: 0.7204 - accuracy: 0.5329 - val_loss: 0.6699 - val_accuracy: 0.5822
Epoch 2/2
146/146 [==============================] - 69s 470ms/step - loss: 0.6542 - accuracy: 0.6131 - val_loss: 0.6731 - val_accuracy: 0.5786


In [ ]:
# evaluate the model
_, train_acc = model_embed.evaluate(train_X, train_Y, verbose=0)
_, test_acc = model_embed.evaluate(test_X, test_Y, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))

# Transformer pretraining